In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [518 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,266 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,337 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,244 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,156 kB

In [ ]:
def getDiscrption(href):
  driver_temp = webdriver.Chrome(options=options)
  driver_temp.get(href);
  time.sleep(5);
  description_div = driver_temp.find_element(By.XPATH,'//*[@id="app"]/div[1]/div[1]/div/main')
  description = ''
  for des in description_div.find_elements(By.CLASS_NAME,'content'):
    description = description + des.text
  driver_temp.quit()
  # print(description);
  return description;

def getCountriesList():
  driver_temp = webdriver.Chrome(options=options)
  driver_temp.get('https://www.scholarshipportal.com/scholarships');
  time.sleep(2);
  list_coun = driver_temp.find_elements(By.XPATH,'//*[@id="app"]/div[1]/div/div[1]/div/div/ul/li');
  Main_List = []
  for a in list_coun:
    link = a.find_element(By.TAG_NAME,'a').get_attribute('href')
    Main_List.append(link)
  driver_temp.quit()
  return Main_List

def getNoOfPages(link):
  driver_temp = webdriver.Chrome(options=options)
  driver_temp.get(link);
  time.sleep(2);
  _coun = driver_temp.find_element(By.XPATH,'//*[@id="app"]/div[1]/div/div[2]/div/h1');
  length = int(int(_coun.text.split(' ')[0])/15);
  driver_temp.quit()
  return length

In [ ]:
import time
import pandas as pd
from selenium.webdriver.common.by import By
new_Data = []
# open it, go to a website, and get results
for c_link in getCountriesList():
  # print(c_link)
  length = getNoOfPages(c_link);
  print('Total pages',length)
  country = c_link.split('/')[-1]
  driver = webdriver.Chrome(options=options)
  URL= c_link+'?page='
  count=0
  new_df=pd.DataFrame()
  for i in range(1,length+1):
      driver.get(URL+ str(i))
  # path= r'E:\FYP\chromedriver_win32\chromedriver.exe'
  # driver=webdriver.Chrome(executable_path=path)
  #driver.get("https://www.scholarshipportal.com/scholarships?page=3")

      time.sleep(5)
      sch_title=driver.find_elements(By.XPATH,"//div[@class='md:w-3/4']//a//div[@class='mb-4']/h3")
      href=driver.find_elements(By.XPATH,"//div[@class='md:w-3/4']//a")
      degree=driver.find_elements(By.XPATH,"//div[@class='md:w-3/4']//a//div[2]/div[2]/span")
      date=driver.find_elements(By.XPATH,"//div[@class='md:w-3/4']//a//div[2]/div[4]/span")
      # print(href)
      r=1
      templist = []
      time.sleep(2)
      #test=pd.DataFrame()

      for title,href,degree,apply_date in zip(sch_title,href,degree,date):
          # try:
              try:
                  sch_title=title.text
              except Exception as ex:
                  sch_title="-"
              try:
                  date=apply_date.text
              except Exception as ex:
                  date="-"
              try:
                  href=href.get_attribute('href')
                  description = getDiscrption(href);
              except Exception as ex:
                  href="-"
              try:
                  degree=degree.text
              except Exception as ex:
                  degree="-"

              count+=1
              Table_dict={ 'Title': sch_title,
                          'Links':href,
                          'Country':country,
                          'degree':degree,
                          'Apply Date':date,
                          'Description': description
                          }

              templist.append(Table_dict)
              #df = pd.DataFrame(templist)
              new_Data.append(Table_dict)
              # new_df.append(Table_dict,ignore_index=True)
              print(len(new_Data))
              r =r+1

          # if there are no more table data to scrape
          # except :
          #     break
          #df.to_csv('studyportal.csv')

      i=i+1
      new_df = pd.DataFrame(new_Data)
      new_df.to_csv('studyportal.csv')
      #print(df)
      #saving the dataframe to a csv

  #print(df)
  driver.close()






WebDriverException: ignored